<h1> Research iteration 2</h1>

<i>Sjoerd Beetsma, Maarten de Jeu
Class V2A - Group 5</i>

<h3>Introduction</h3>

Like mentioned in research iteration 1, we have divided our 3 research questions across 3 datasets. The first 2 research questions correspond to the <i>Chemical</i> datasets, and the last questions corresponds to the <i>review</i> dataset.

Throughout this notebook, we'll attempt to stick to the iterative CRISP-DM workflow as much as possible, and divide our notebook into chapters corresponding to phases from this philosophy. This chapter division is only a rough indication of what can be found where though, because the highly iterative workflow requires us to look back/forward in the process quite frequently.

Specifically, everything you're looking for can be found in this order:

<ol>
<li>Business understanding: Chemical dataset.</li>
<li>Data understanding: Chemical dataset.</li>
<li>Data preparation: Chemical dataset.</li>
<li>Modelling: Research question 1.</li>
<li>Modelling: Research question 2.</li>
<li>Business understanding: Review dataset.</li>
<li>Data understanding: Review dataset.</li>
<li>Data preparation: Review dataset.</li>
<li>Modelling: Research question 3.</li>
</ol>

TODO(m-jeu): Evaluation after every modelling phase? Or combine into 1 chapter perhaps?

<h2>Business Understanding: Chemical dataset.</h2>

The context for our research questions is quite vague from the side of our client, but we'll be examining the following questions:

<ol>
<li>With what accuracy can we decide the quality of a red-wine according to its chemical properties.</li>
<li>Can we predict a wine's color based on it's chemical properties?</li>
</ol>



<h2> Data Understanding: Chemical datasets.</h2>

The dataset for the first research question is aquired from https://archive.ics.uci.edu/ml/datasets/wine+quality along a dataset about red-wine there is also a dataset about white-wine, the first will be used for our first research question and the datasets combined will be used for our second research question.

The variables in the chemical datasets are:<br />

1 - fixed acidity: Fixed acids found in wines are tartaric, malic, citric, and succinic.   <br />
2 - volatile acidity: Steam distillable acids like acetic acid, lactic, formic, butyric, and propionic acids.  <br />
3 - citric acid: One of the fixed acids found in a wine <br />
4 - residual sugar: Residual sugar in a wine is the natural grape sugars leftover in a wine after the fermentation is finished. <br />
5 - chlorides: Chlorides are a major conributor to the saltiness of a wine.<br />
6 - free sulfur dioxide:The part of sulfur that remains free after the sulfur is binded <br />
7 - total sulfur dioxide:Free sulfur dioxide plus binded sulfur dioxide. <br />
8 - density: Density of a wine, usually increased by fermentable sugars. <br />
9 - pH: pH to specify the acidity/basicity of a wine, most wines ranging with a pH from 3 to 4. <br />
10 - sulphates: Use of sulpates slows down the growth of yeasts keeping and prevents the oxidation keeping the wine fresh for longer. <br />
11 - alcohol: Alcohol percentage of a wine <br />
12 - quality: A quality score between 0 and 10, based on sensory data <br />

The source of the data also states that they don't know if all variables are relevant in deciding the quality score of a wine.

#todo add source of data


We import some libraries and the dataset to examine the data through code.

In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk

# FIXME(m-jeu): Is this necessary. Ik stel voor dat we allebei individueel ons gebruik van dingen hieruit omschrijven naar sk.nogwat.nogwat
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import cluster
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn import metrics
from sklearn import naive_bayes

import plotly.express as px

import py_lib
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

For research question one we will only need the red-wine dataset. The second research question requires a combined dataset with labels added for the color of the wine (white or red) the two datasets will be explored and cleaned at the same time but separately, performing similar operations on both. Merging the two datasets before the data-cleaning would result in records being designated outliers that shouldn't be, correlations not being detected in certain datasets that should be, etc. We'll merge the two datasets after cleaning them for use in the second research question.

Let's load in both red and white wine datasets

In [ ]:
dataset_red = pd.read_csv("datasets/winequality-red.csv", sep=";")
dataset_white = pd.read_csv("datasets/winequality-white.csv", sep=";")

Let's take the head of one of the chemical property datasets to have a first look at the submissions of the data.

In [ ]:
dataset_red.head()

As described by the source each row seems to correspond with a individual wine. With eleven different feature columns describing the chemical properties of the wine and one target column, quality.

To access columns easier in the future change white spaces in column names to underscores.

In [ ]:
dataset_red.columns = dataset_red.columns.str.replace(' ','_')
dataset_white.columns = dataset_white.columns.str.replace(' ','_')

To see howmuch data entries we have we will check the amount of rows in both the red and white datasets.

In [ ]:
red_rows, red_columns = dataset_red.shape
white_rows, white_columns = dataset_white.shape

print(f'The red-wine quality dataset contains {red_rows} rows and {red_columns} columns')
print(f'The white-wine quality dataset contains {white_rows} rows and {white_columns} columns')

Lets change the column name white spaces to underscores to make life easier.

In [ ]:
dataset_red.head(0)

<h3>Target and feature variables</h3>

For research question 1, all the columns describing chemical properties will be considered as a feature variable and the column quality represents the target variable, the variable we want to predict.
Lets safe them in a variables to access them easily from the final dataframe.

The feature variables (at the start) for research question are the same ones as those for question 1, but will change after some investigation, and the target variable is currently not available as column (the eventual target variable will be whichever dataset the record is in now).

In [ ]:
feature_vars_q1 = ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'pH', 'sulphates', 'alcohol']
target_var_q1 = 'quality'

<h3> Scales of measurements </h3>

To choose a appropriate model for our research-questions and available data it's necessary to have a understanding of all the scales of measurements for all the relevant variables.

In [ ]:
nomi, disc, ordi, cont = 'Nominal', 'Discrete', 'Ordinal','Continous'
print('Scales of measurement chemical properties datasets')
pd.DataFrame(index=dataset_red.columns, data=[cont, cont, cont, cont, cont, cont, cont, cont, cont, cont, cont, disc],
             columns=['Scale of measurement'])

As can be seen all the chemical properties have continous scale of measurement and the quality columns has a Discrete scale of measurement.

<h3>Central tendancies and dispersion measures</h3>

Using describe we can see some important central tendancies and dispersion measures about the dataset.

In [ ]:
dataset_red.describe().round(2)

From the describe we can tell that there are quite a few columns with a big difference between maximum and minimum values which indicate outliers. <b>TODO(m-jeu): Does this indicate outliers? If std is large aswell, it doesn't have to right?</b>

The columns with big differences between max and min values:
Residual_sugar, chlorides, free_sulfur_dioxide ,total_sulfur_dioxide.

In [ ]:
dataset_white.describe().round(2)

Just like the red-wine dataset, the white-wine dataset has similair differences in maximum and minimum values.

Lets take a more visual look at the distribution of all data through a histogram for each of the feature attributes
Starting off with the red wine dataset.

In [ ]:
dataset_red.hist(figsize=(15,15))
plt.show()

In the the red-wine dataset pH and possibly density have a gaussian distribution. Other variables seem to have a skewed distribution.<br/>
Moving on to the white-wine dataset.

Moving on to the white wine dataset:

In [ ]:
dataset_white.hist(figsize=(15,15))
plt.show()

In the the white-wine dataset only pH seems to have a gaussian distribution maybe density aswell after removing outliers. Other variables have a skewed, possibly lognormal, distribution.<br/>

In [ ]:
sns.countplot(data=dataset_red, x='quality').set_title('red-wine quality counts')
plt.show()
sns.countplot(data=dataset_white, x='quality').set_title('white-wine quality counts')
plt.show()

From the count plots above we can tell the quality for red-wines range from 3 to 8 with 5 being the most common quality rating. For white-wines it ranges from 3 to 9 with 6 being the most common quality rating.

<h3>Outliers</h3>

To get a visual understanding of the outliers in the feature columns each feature gets a boxplotted with the Q1 target variable quality. Giving a small summary of the minimum, Q1, Q2 (median), Q3 and the maximum of each attribute plotted against quality scored to give a view of outliers at all quality levels.

In [ ]:
def boxplotter(dataset, y_axes, x_axis):
    """Function to boxplot 1 x_axis against a list of y_axis of a given dataset"""
    for col in y_axes:
        sns.boxplot(x=dataset[x_axis], y=dataset[col])
        plt.show()

First boxplot all the feature variables on the y-axis against the Q! target variable on the x-axis for the red-wine dataset.

In [ ]:
boxplotter(dataset=dataset_red, y_axes=feature_vars_q1, x_axis=target_var_q1)

Now do the same for the white-wine dataset

In [ ]:
boxplotter(dataset=dataset_white, y_axes=feature_vars_q1, x_axis=target_var_q1)

As can be seen from the boxplots all of our current variables contain outliers.

All outliers in the above boxplots seem to be plausible and not from incorrect data, like some attributes in iteration 1 had.
From the boxplot with alcohol on the y axis and quality on the x axis we can  see that a trend of a rising median alcohol percentage the higher the quality of the wine.

<h3>Correlations</h3>

For later models it's important to know what variables have a (linear) correlation between each other.
To find linear correlations and their direction/strength we make use of Pearson's correlation coefficient.

In [ ]:
def corr_matrix_plotter(dataset, title=''):
    """Return a correlation matrix created using seaborn and matplotlib that for all columns in
    a pandas dataframe.

    Args:
        dataset: Dataset to construct correlation matrix for.
        title: Title of the plot.

    Returns:
        correlation matrix."""
    corr = dataset.corr()
    plt.figure(figsize=(10,7.5))
    cmap = sns.diverging_palette(200, 0, as_cmap=True) # color palette as cmap
    mask = np.logical_not(np.tril(np.ones_like(corr))) # triangle mask
    sns.heatmap(corr, annot=True, mask=mask, cmap = cmap, vmin=-1, vmax=1).set_title(title) # correlation heatmap
    plt.show()

corr_matrix_plotter(dataset_red, 'Red wine correlation matrix.')
corr_matrix_plotter(dataset_white, 'White wine correlation matrix.')

In the correlation matrices graphed above you can see which attributes have a correlation to other attributes. Starting with Q1 our target variable 'quality', we can see quality has a few correlations with the strongest one being alcohol for both red and white wines and a few weaker ones like volatile acidity, sulphates and citric acid for red wines and density and chloride for white wines. Because quality is our Q1 target variable it's the independent attribute in the correlations.

Quality is dropped for research question 2, so for that it's not relevant.

Besides there are some corelations among chemical properties:
Fixed acidity has strong correlation with pH, but it’s still an independent type. pH However is a dependent type; it depends on the former. Volatile acidity, residual sugar, sulphates, chlorides, and density are all independent data types. Total sulfur dioxide is dependent on free sulfur dioxide, but free sulfur dioxide is independent.

Strong correlations along features might need to be removed during the data preparation phase.

<h3>Data Preparation: Chemical dataset.</h3>

Lets start of the data preparation by checking the datatypes and clean or change them if necessary.

Red-wine quality datatypes:

In [ ]:
dataset_red.dtypes

White-wine quality datatypes:

In [ ]:
dataset_white.dtypes

These all seem to be in order, so we can now move on to checking and removing or replacing any NA values in the datasets.

In [ ]:
pd.isna(dataset_red).sum().sum() # checking for total NA values in red_wine

In [ ]:
pd.isna(dataset_white).sum().sum() # checking for total NA values in white_wine

But luckily, all data seems to be complete, so no need to worry about that either.

<h4>Removing outliers</h4>

A remove outliers function is created but and used to remove extreme outliers.
This is will make a regression algorithm peform better because the RMSE is sensitive to outliers.

Removing all extreme the outliers leaving the mild ones in the dataset with a outer fence.
3 * IQR below Q1 or 3 * IQR above Q3.

In [ ]:
def remove_outliers(dataset, fence = 3):
    """
    Function to remove any rows containing one or more outliers in a dataframe.
    Default fence of 3 * IQR detecting extreme outliers.

    args:
        dataset: Pandas Dataframe or Series
        fence: Number deciding the fence range to use to detect outliers.
    
    returns:
        The passed dataset minus the rows containing outliers
    """
    q1 = dataset.quantile(.25)
    q3 = dataset.quantile(.75)
    iqr = q3 - q1
    return dataset[(dataset >= q1 - (fence * iqr)) & (dataset <= q3 + (fence * iqr))].dropna() # turn extreme outliers into NaN values and drop the rows

The red-wine dataset contained 1599 rows and the white-wine 4898 before removing the outliers lets remove the outliers and check how many are left.

In [ ]:
dataset_red = remove_outliers(dataset_red)
dataset_white = remove_outliers(dataset_white)

In [ ]:
dataset_red.shape, dataset_white.shape

1435 of the 1599 rows are left in the red-wine data, 12% of the rows contained etreme outliers.<br/>
And in the white-wine data 4690 of the 4898 rows are left, 4% of the rows contained extreme outliers.<br/>



<h3>Duplicates</h3>

Duplicate values would probably cause overfitting in the worst case, and imbalanced models in the best case. Let's check if either dataset contains any.

In [ ]:
dataset_white.duplicated().sum()

In [ ]:
dataset_red.duplicated().sum()

Both contain duplicates that need to be removed, so let's get rid of them.

In [ ]:
dataset_white.drop_duplicates(inplace=True)
dataset_red.drop_duplicates(inplace=True)

The red and white wine chemical property datasets are cleaned but still needs normalizing. Because we don't need normalized data for our first research question and only for our second one we will only normalize the red and white combined dataset.

Now both the red and white dataset are ready to be combined into one dataset with a extra column for being red or not.



In [ ]:
dataset_red['is_red'] = 1
dataset_white['is_red'] = 0
dataset_red_white = pd.concat([dataset_red, dataset_white])

In [ ]:
dataset_red_white.sample(10,random_state=2)

<h4>Normalizing data</h4>

Machine learning algorithms using a euclidean distance function benefit from working with normalized data.<br/>
We'll define a function that normalizes a pandas Series or Dataframe object. We won't use it for now, because it best to only do it when needed, but it's good to have.

In [ ]:
def normalize(s: pd.Series or pd.DataFrame) -> pd.Series or pd.DataFrame:
    """Normalize (standardize) a pandas series or dataframe object by subtracting the mean and dividing by the
    standard deviation.

    Args:
        s: series object to normalize.

    Returns:
        normalized series object."""
    return (s - np.mean(s)) / np.std(s)

<h4>Data cleaned</h4>

We now have the dataset_red, dataset_white, and dataset_red_white Dataframes, which are all cleaned. Nothing is normalized as of yet, but we have defined a procedure to do this with.

<h3>Modeling: Chemical datasets</h3>

<h4>Test and train data</h4>

The datasets will be splitted into a train and test dataset for the models to learn and test their performance.


In [ ]:
X, y = dataset_red[feature_vars_q1], dataset_red[target_var_q1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

scores = pd.DataFrame()

<h4>Baseline model</h4>

We start out by creating a baseline model that always predicts the mean of the target variable in the training set, so that we can attempt to improve this score with other models. Let's check the RMSE of this model.

In [ ]:
baseline = py_lib.DumbRegressor(y_train)
base_line_predictions = baseline.predict(X_test)

In [ ]:
plt.plot(base_line_predictions, c='r')
plt.scatter(x=np.arange(len(X_test)), y=y_test, s=4)
plt.xlabel("Testset measurement")  # FIXME(m-jeu): Better xlabel?
plt.ylabel("Quality score")
plt.legend(["Baseline prediction", "Actual value"], loc="upper left")
plt.show()

In [ ]:
baseline_rmse = sk.metrics.mean_squared_error(y_test, base_line_predictions, squared=False)
scores = scores.append({'algorithm':'baseline', 'RMSE': baseline_rmse},ignore_index=True)

In [ ]:
scores

The baseline model scores a RMSE of .82 quality points, this score is what we want to improve upon with further models.

<h4>Implementing a machine learning model</h4>

For our first machine learning model we will implementing the simpelest form of multiple feature regression: multiple linear regression.
This model doesn't have any hyper-parameters to tune.

In [ ]:
# multiple linear regression
lin_regr = linear_model.LinearRegression()

Fit the model to the X train and y train data

In [ ]:
lin_regr.fit(X_train, y_train)

Get the mean_squared_error to see how good the prediction is vs the actual values.

In [ ]:
#################################
# FIXME(m-jeu): Volgens mij is er vanaf hier iets mis gegaan in het kopieren, dus daar moeten we mss morgen nog ff naar kijken.
# Ik heb y_test_red naar y_test veranderd, omdat ik dacht dat die 2 redundant van elkaar waren, maar mss was dat wel dom van me.
# Ik pas vanaf hier tot research question 2 vanavond even niks meer aan.
#################################


Lets see how multiple linear regression compares to the baseline model. 
Get the RMSE to decide the peformance of the linear model.

In [ ]:
lin_regr_model_score = sk.metrics.mean_squared_error(lin_regr.predict(X_test), y_test, squared=False)
scores = scores.append({'algorithm':'multiple linear regression', 'RMSE': lin_regr_model_score},ignore_index=True)
lin_regr_model_score

The multiple linear regression scored a RMSE 0.61 compared to our baseline this is a 26% lower RMSE, meaning the baseline has already been improved surpassed by the first model. But it's not quite as low as we want it yet. Lets try to improve more with polynomial regression. Through polynomial regression we can see if the number of polynomials will result in a better peformance. A polynomial model with just 1 degree is the same as linear regeression.

For our hyper-parameter, the degree of polynomials we will be trying 1 to 7 degrees.

In [ ]:
# polynomial regression
poly_r_degrees = np.arange(1,8,1)

Make the model

In [ ]:
poly_r_make_train_model = np.vectorize(lambda d: make_pipeline(PolynomialFeatures(d),linear_model.LinearRegression()).fit(X_train, y_train))

Train a model for every degree earlier specified degree.

In [ ]:
poly_r_models = poly_r_make_train_model(poly_r_degrees)

Lets see how well the polynomial regression peformances with different degrees of polynomials.

In [ ]:
poly_r_models_scores = np.vectorize(lambda m: sk.metrics.mean_squared_error(m.predict(X_test), y_test, squared=False))(poly_r_models)
for degree, score in enumerate(poly_r_models_scores):
    print(f"degree {degree+1}, score {score}")
scores
scores = scores.append({'algorithm':'Polynomial regression', 'RMSE': np.min(poly_r_models_scores)},ignore_index=True)
plt.plot(poly_r_degrees, poly_r_models_scores, 'bx-')
# Add title and axis names
plt.title('Polynomial degree VS RMSE score')
plt.xlabel('Polynomial degree')
plt.ylabel('RMSE')
plt.show()

Polynomial regression peformed best with a polynomial degree of 1. This means polynomial regression won't have any benefit over linear regression.

Lets also test ridge and lasso regression, these regression algorithms are less sensitive to correlations among feature variables.

For Ridge regression we'll be trying out alphas #to-do.
And for Lasso regression the alphas #to-do will be tested, since Lasso regression doesn't work with a alpha of 0.

#---

In [ ]:
# ridge regression
ridge_r_alphas = np.arange(0,10,1)

Create different a vectorized function to create a Ridge model with a passed alpha setting, fitted to the train data.

In [ ]:
ridge_r_make_train_model = np.vectorize(lambda a: Ridge().set_params(alpha=a).fit(X_train, y_train))

Make the models with the above function and the ridge alphas from 0 to 10 with steps of 1.

In [ ]:
ridge_r_models = ridge_r_make_train_model(ridge_r_alphas)

Test the models their peformance by predicting on the test dataset getting the RMSE from the prediction and the actual values.

In [ ]:
ridge_r_models_scores = np.vectorize(lambda m: sk.metrics.mean_squared_error(m.predict(X_test), y_test, squared=False))(ridge_r_models)
plt.plot(ridge_r_alphas, ridge_r_models_scores, 'bx-')
plt.show()
print(min(ridge_r_models_scores))

Ridge regression seems to peform better by a very small margin.

Lastly lets try Lasso regression.

Alphas that will are 0.001 to .1 with steps of 0.01

In [ ]:
#lasso regression
lasso_r_alphas = np.arange(0.01, 1, 0.01) # Lasso doesn't start at 0

Make the model

In [ ]:
lasso_r_make_train_model = np.vectorize(lambda a: Lasso().set_params(alpha=a).fit(X_train, y_train))

Train the model with the different alphas.

In [ ]:
lasso_r_models = lasso_r_make_train_model(lasso_r_alphas)

Get the RMSE to decide the peformance of the linear model

In [ ]:
lasso_r_models_scores = np.vectorize(lambda m: sk.metrics.mean_squared_error(m.predict(X_test), y_test, squared=False))(lasso_r_models)
plt.plot(lasso_r_alphas, lasso_r_models_scores, 'bx-'), plt.show()
print(min(lasso_r_models_scores))

Scatterplot of the two most correlating columns and the predicted values

In [ ]:
#px.scatter_3d(x=X_test, y=X_test, z=y_predictions)

<h3>Conclusion first model</h3>

Conclusion to be added. First model requires more tuning and adjusting before a final conclusion

<h3>Modeling: research question 2.</h3>

<h4>Test and train data</h4>

For convenience, we'll add an overview of the 'combined' dataset. Then we'll split the dataset into a test and train portion. As a reminder: we'll be using the chemical properties as feature variables to try to determine whether a wine is red of white from these. The target variable is 'is_red', a nominal(boolean) variable with 2 possible values. We'll also standardize the feature variables, because we're using a model with euclidean distance measurements after all.

In [ ]:
dataset_red_white

In [ ]:
X, y = normalize(dataset_red_white[feature_vars_q1]), dataset_red_white['is_red']
X_train_red_white, X_test_red_white, y_train_red_white, y_test_red_white = train_test_split(X,y, random_state=0)

<h4>Baseline model</h4>

We'll start out by creating a simple model that always guesses the mode measurement of the target variable in the train dataset, and seeing what accuracy that gets, so that we have a score to attempt to improve upon.

In [ ]:
baseline = py_lib.DumbNominalClassifier(y_train_red_white)
baseline_predictions = baseline.predict(X_test_red_white)
baseline_predictions[:2]

The majority of the test dataset consists of white wine, so that's what we'll blindly be guessing with the baseline model.

In [ ]:
metrics.accuracy_score(y_test_red_white, baseline_predictions)

The baseline model is already able to get an accuracy of +/- 77.21% on the test dataset.

We'll try to improve on this, starting with a simple model and moving towards more complex ones. We'll start out using KNearestNeighbors because of it's simplicity, but because of the over-representation of white wine, we suspect other models might be able to do better.

Normally, one would build in business-knowledge to pick a few variables that might be relevant to start out with. In our case, we have no business expert, so we'll start with many attributes, and eliminate any unnecessary ones along the way.

Hyper-parameter wise, we'll start by experimenting with different values for 'k'. Our classes are way bigger, but we'll call the max sensible value for k 100 for now, starting at 1, with any odd values being fair game because we only have 2 target classes.

In [ ]:
def knn_models(k_vals: np.ndarray,
               X_train: pd.DataFrame,
               y_train: pd.Series,
               X_test: pd.DataFrame,
               y_test: pd.Series) -> pd.DataFrame:
    """Create, fit and score KNN-models with different k-values for one dataset parallel to each other
    through numpy vectorized operations.

    Args:
        k_vals: values for k to train a model with.
        X_train: feature variables training set.
        y_train: target variables training set.
        X_test: feature variables test set.
        y_test: target variables test set.

    Returns:
        Pandas dataframe that contains:
            index column with every k-value.
            'Model' column with the associated model.
            'Score' column with the accuracy score that model got against the test-set."""
    models = np.vectorize(lambda k: neighbors.KNeighborsClassifier(n_neighbors = k))(k_vals)   # Initialize model objects.
    models = np.vectorize(lambda m: m.fit(X_train, y_train))(models)                           # Train models.
    scores = np.vectorize(lambda m: metrics.accuracy_score(y_test, m.predict(X_test)))(models) # Make predictions for test set and score.
    return pd.DataFrame({"Model": models,                                                      # Create and return Dataframe.
                         "Score": scores},
                        index=k_vals)


In [ ]:
ks = np.arange(1, 101, 2)
models = knn_models(ks, X_train_red_white, y_train_red_white, X_test_red_white, y_test_red_white)
plt.plot(models.index, models["Score"], 'bx-')

In [ ]:
X_test

It appears (k < 20 ∧ k > 0) is the most promising area, so let's have a look at the exact values.

In [ ]:
models.loc[:20, "Score"]

We're getting above 99% accuracy, with the peak around 99.60%. Out of all the choices with peak accuracy, it's good to pick a higher setting for k to avoid overfitting. k = 9 has this top score for instance, and would probably be a good choice for production.

99.6% is an incredible score, which would mean a model is terribly useful. So incredible, that we should probably consider the fact that something might have gone wrong. We split our data into a training and a test test:

In [ ]:
print(f"""Data shapes:
   Train      Test
X: {X_train_red_white.shape} {X_test_red_white.shape}
y: {y_train_red_white.shape}    {y_test_red_white.shape}""")

We removed duplicates that could end up in both the train and the test dataset from the dataset in data preparation.

Perhaps certain chemical values really do have big discernible gaps between red and white wine. We know that most attributes have normal and lognormal distributions in either the red or the wine dataset individually, but one would expect there to be obvious differences between the distributions in either the red or wine dataset respectively.

In [ ]:
columns = dataset_white.loc[:, :"alcohol"].columns  # We don't need to look at quality or is_red.
fig = plt.figure(figsize=(15, 15))
fig.subplots_adjust(hspace=0.4)
fig.suptitle("Attribute value distributions in Red / White dataset", fontsize=16)

for i, col in enumerate(columns):
    white_view, red_view = dataset_white[col].to_numpy(), dataset_red[col].to_numpy()  # View of Numpy arrays contained in relevant cols.

    # To enforce equal bins size between 2 histograms.
    att_min = min(np.min(white_view), np.min(red_view))
    att_max = max(np.max(white_view), np.max(red_view))
    bins = np.linspace(att_min, att_max, 20)

    #Actually making subplot
    ax = fig.add_subplot(4, 3, i + 1)
    ax.set_title(col)
    ax.set_ylabel("Frequency")
    ax.set_xlabel("Value")
    ax.hist(dataset_white[col], alpha=0.5, bins=bins, color="yellow")
    ax.hist(dataset_red[col], alpha=0.5, bins=bins, color="r")
    ax.axvline(np.mean(white_view), color="darkorange", linestyle="dashed")
    ax.axvline(np.mean(red_view), color="darkred", linestyle="dashed")
    ax.legend(["White mean", "Red mean", "White frequency", "Red Frequency"], loc="upper right")

From this improved we can see that there are indeed some quite descriptive variables when it comes to predicting what color a wine is, which could explain the high accuracy.

We can divide the attributes into 3 categories based on these distributions (based on visual inspection, and no formal math or logic of any kind):

<ol>
<li>Attributes that tell us a lot about the potential color of a wine.</li>
<ul><li>Volatile Acidity</li><li>Chlorides</li><li>Total sulfur dioxide</li></ul>
<li>Attributes that tell us a little, or might tell us a lot about the potential color of a wine</li>
<ul><li>Fixed acidity</li><li>Residual sugar</li><li>Free sulfur dioxide</li><li>Density</li></ul>
<li>Attributes that tell us close to nothing about the color of a wine</li>
<ul><li>Citric acid</li><li>pH</li><li>Alcohol</li><ul>
</ol>

We'll synthesize some new Train/Test feature datasets from the old ones, based on the first category, to train some new models with to see if a stripped down model can perform just as well.

In [ ]:
cat1 = ["volatile_acidity", "chlorides", "total_sulfur_dioxide"]
X_train_red_white_2, X_test_red_white_2 = X_train_red_white[cat1], X_test_red_white[cat1]

In [ ]:
models = knn_models(ks, X_train_red_white_2, y_train_red_white, X_test_red_white_2, y_test_red_white)
plt.plot(models.index, models["Score"], 'bx-')

In [ ]:
models.loc[45:55, "Score"]

We can see that a KNN-model based on only 3 attributes (volatile_acidity, chlorides and total_sulfur_dioxide) can still predict what color a wine is with 99.20% accuracy when k = 47 (in the test set), which could also be interesting for an eventual model in production.

Because we're dealing with so many Gaussian distributions, with distinctive differences between the target categories for some attributes, this problem could also be well suited for a Gaussian Naive Bayes Classifier.

Even though the model we have right now already performs quite well, it could be interesting to try this out in an attempt to reach 100% accuracy. We'll try a Gaussian Naive Bayes classifier with both all the chemical properties, and the stripped down chemical properties.


In [ ]:
nb_model_complete = naive_bayes.GaussianNB()
nb_model_complete.fit(X_train_red_white, y_train_red_white)
nb_model_complete.score(X_test_red_white, y_test_red_white)

In [ ]:
nb_model_stripped = naive_bayes.GaussianNB()
nb_model_stripped.fit(X_train_red_white_2, y_train_red_white)
nb_model_stripped.score(X_test_red_white_2, y_test_red_white)

With 99.1% accuracy with all feature variables, and 97.8% with the stripped down feature variable list, the Gaussian Naive Bayes classifier performs worse then K Nearest Neighbors in both scenarios, which is quite unexpected. A possible real-world advantage of this could be the fact that this classifier could return the probabilities for a given record being in either class, which could be useful.

<h3>Conclusion research question 2</h3>

With the use of a fairly simple algorithm like K Nearest Neighbors, one can easily classify the color of wine based on certain chemical properties with an accuracy above 99%, even when just looking at volatile acidity, chlorides and total sulfur dioxide. Simplicity seems to be key in this example, because a more complex model like a Gaussian Naive Bayes classifier performs every so slightly worse.

<h4>Implementing a machine learning model</h4>

TODO(m-jeu): Actually move these headers to the proper place lmao.


<h4>Hyper-parameters</h4>

<h4>Validation</h4>

<h3>Research question 3</h3>


<h2> Business understanding </h2>

For our third research question we have a dataset about wine reviews from different smelters with information about the origin of the wine, price and their review.
<br/>
With our research question being:
<i>Can we distinguish between logical clusters of wineries? (Premium, budget, high-quality, etc...)</i>


<h2> Data Understanding Wine-review dataset.</h2>


From our dataset's source ..., we have a list of the different attributes:  # FIXME(m-jeu): Source.
#FIXME(m-jeu): Do we need to describe these?

1 - country<br/>
2 - description<br/>
3 - designation<br/>
4 - points<br/>
5 - price<br/>
6 - province <br/>
7 - region_1<br/>
8 - region_2<br/>
9 - taster_name<br/>
10 - taster_twitter_handle<br/>
11 - title <br/>
12 - variety<br/>
13 - winery<br/>

Let's load in the dataset and have a first look.

In [ ]:
dataset_reviews = pd.read_csv("datasets/winemag-data-130k-v2.csv")
dataset_reviews.head(5)

At first look it appears most of the (usable) variables are nominal, with points and price as the only numerical (discrete) values. We have some columns that initially seem fairly useless for the types of analysis that we will most probably be using for this project, like 'description', but we'll keep them just in case we end up doing anything like a sentiment-analysis type model. It also appears we have a redundant index columns called "Unnamed: 0".

<h4>Target and feature variables</h4>

We're not quite sure what feature variables we'll be using for the third question, but we know we'll be grouping by 'winery'. We'll start out by using 'price' and 'points' as further feature variables, but during the modelling stage we might end up using more.

Considering we're looking for logical clusters (unsupervised learning), there are no target variables.

<h4>Scales of measurement</h4>

Like mentioned earlier, we're mostly dealing with categorical (ordinal, specifically) variables in this dataset. There are 2 numerical values. Points and price are both discrete values.

<h4>Central tendencies and dispersion measures</h4>

We can examine the spread of values of the numerical variables through histograms:

In [ ]:
_ = dataset_reviews[["price", "points"]].hist()  # _ = to prevent pointless table from showing on screen.

Points has an obvious Gaussian distribution. It does appear the price graph is made quite unreadable by some outliers. We'll have a proper look at those later, let's ignore them for now to have a better look at the distribution.

In [ ]:
dataset_reviews[dataset_reviews["price"] < 100]["price"].hist(bins=20)

It appears that the price column in lognormally distributed.

<h4>Outliers</h4>

Because we won't be comparing against something, we'll be using a seperate way of creating boxplots to explore outliers for the numeric values.

In [ ]:
sns.boxplot(data=dataset_reviews[["points"]])

It appears the median' amount of points is around 88, and quite symmetric. There are a couple of outliers around 100, but nothing extreme.

In [ ]:
sns.boxplot(data=dataset_reviews[["price"]])

The boxplot for price is barely a boxplot because of all the outliers. Like we already noticed with the histogram, most measurements fall within the 0-100 range, but there are some extremely high outliers. For data exploration, we'll create a separate column with the outliers removed for price.

In [ ]:
dataset_reviews["price_no_outliers"] = remove_outliers(dataset_reviews["price"])  # FIXME(m-jeu): remove_outliers drops na
dataset_reviews["points_no_outliers"] = remove_outliers(dataset_reviews["points"])# Which then get filled in again...

<h3>Correlations</h3>

We'll use the pearson correlation coefficient to see if there's a linear correlation between the only 2 numerical columns in the dataset. Because of the extreme outliers in price, it might be sensible to also check this with the outliers removed.

In [ ]:
dataset_reviews[["points", "price"]].corr()

In [ ]:
dataset_reviews[["points_no_outliers", "price_no_outliers"]].corr()

It appears there is a weak linear correlation between price and points, when disregarding outliers. Because we're looking for cluster, this is not terribly relevant, but still noteworthy.

<h2>Data preparation: Review dataset</h2>

Having explored the data, we're ready to clean it up. We have already separated the outliers in a separate column in the dataset, because that couldn't wait until data preparation. First, let's get rid of the unnamed index column, because that's not useful in any way.

In [ ]:
dataset_reviews.drop("Unnamed: 0", axis=1, inplace=True)
dataset_reviews.dtypes

And let's change the categorical values from 'object', to 'string', to 'category'.

In [ ]:
dataset_reviews = dataset_reviews.convert_dtypes()

categorical_vars = ["country", "description", "designation", "province", "region_1", "region_2", "taster_name", "taster_twitter_handle", "title", "variety", "winery"]

dataset_reviews[categorical_vars] = dataset_reviews[categorical_vars].astype("category")
dataset_reviews.dtypes

<h2>Modelling: Review dataset</h2>

Our goal is to find logical clusters for different types of wineries, using a clustering algorithm. First, we create a separate dataframe from the original dataframe grouped by winery, with the mean value of the point/price value of that winery's wine as columns.

In [ ]:
dataset_winery = dataset_reviews.groupby("winery")[["price_no_outliers", "points_no_outliers"]].mean()
dataset_winery.dropna(inplace=True)
dataset_winery

For many clustering algorithms, it's useful to have the data normalized as well. Let's do that in seperate columns.

In [ ]:
dataset_winery["price_normalized"] = normalize(dataset_winery["price_no_outliers"])
dataset_winery["points_normalized"] = normalize(dataset_winery["points_no_outliers"])

Let's have a look at the data:

In [ ]:
plt.scatter(dataset_winery["price_normalized"], dataset_winery["points_normalized"], s=2)

So far, it mostly just looks like a thick cloud. Hopefully, a clustering algorithm will be able to see through the fog and give us more insight.

We'll start out by using kMeans on the (normalised) points and price because of it's simplicity, and move onto using more complex models and/or adding more data in case it doesn't give any useful results.
Even though it's doubtful anything with k > 20 will be useful, we'll still try everything from k=2 to k=30, to see how the model behaves.

In [ ]:
ks = np.arange(2, 31)  # Values for k to try out.

models = np.vectorize(lambda k: cluster.KMeans(n_clusters=k, random_state=0))(ks)  # Create model objects.
models = np.vectorize(lambda m: m.fit(dataset_winery[["price_normalized", "points_normalized"]]))(models)  # Fit models.
c1_models_scores = np.vectorize(lambda m: m.score(dataset_winery[["price_normalized", "points_normalized"]]))(models)  # Score models
plt.plot(ks, c1_models_scores, 'bx-')

Considering the fact there's no clear 'elbow' in the elbow plot, and no clear clusters in the previously constructed graph, it doesn't look very promising. We can look at a visualization at k=6
of the result to confirm:

In [ ]:
final_model =models[4]  # Take it out of array for convenience.
predictions = final_model.predict(dataset_winery[["price_normalized", "points_normalized"]])
plt.scatter(dataset_winery["price_no_outliers"], dataset_winery["points_no_outliers"], c=predictions, s=2)

Like we expected, not very useful. It's doubtful another algorithm would be able to make sense out of that cloud, so the way to go is probably to attempt to use the curse of dimensionality to our advantage.
There is one problem: there isn't much numerical data available to use in our models. All the categorical values in the dataset are nominal, so that would mean using a get_dummies() construction to turn them into useful data. Unfortunately, all the nominal variables have too many permutations to realistically use, so we'll have to get creative.

In interesting metric could be the amount of wine reviews listed for a single winery. We'll create a new grouped by dataframe, with this column added, normalize it's columns, and visualize it.

In [ ]:
dataset_reviews["winery"].value_counts()

In [ ]:
dataset_winery_2 = dataset_reviews.groupby("winery").agg({"price_no_outliers": np.mean,
                                                          "points_no_outliers": np.mean,
                                                          "title": lambda np_a: np_a.size})  # Title is a random column
                                                                                             # Doesn't really matter what column
                                                                                             # We pick here
dataset_winery_2.rename(columns={"title": "review_amount"}, inplace=True)  # Rename columns to better name

dataset_winery_2.head(5)

In [ ]:
dataset_winery_2["price_normalized"] = normalize(dataset_winery_2["price_no_outliers"])
dataset_winery_2["points_normalized"] = normalize(dataset_winery_2["points_no_outliers"])
dataset_winery_2["reviews_normalized"] = normalize(dataset_winery_2["review_amount"])
dataset_winery_2.dropna(inplace=True)

In [ ]:
fig = px.scatter_3d(data_frame=dataset_winery_2,
              x='price_normalized',
              y='points_normalized',
              z='reviews_normalized')

fig.update_traces(marker={'size': 1, 'colorscale': 'Viridis', 'opacity': 0.8})

<h3>Conclusion research question 3.</h3>

Visually, we can see there's still no sensible clusters to be found. There is no more numerical data, and creating even more from what we have would be fairly pointless. Unfortunately, we have to conclude that there's no useful insight to be gained from clustering on our current data, as there just aren't many numbers to work with.